In [1]:
# %%writefile Ind_xxxxx.py

import sys
import os 
module_path = os.path.abspath(os.path.join('..')) 
if module_path not in sys.path: 
    sys.path.append(module_path)
    
import numpy as np
import pandas as pd


import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

import tools.Sample_Tools as smpl
import tools.Pretreat_Tools as pretreat


In [ ]:
# Amihud定义了ILLIQ度量
# 一个粗略而实用的价格影响指标考虑了一段时间内绝对股票收益率与美元成交量的日平均比率。遵循Kyle的流动性概念，
# ILLIQ = 1/n * sum_t(|r_t|/v_t)

In [ ]:
# 均衡度量
# 基于均衡价格的测度试图捕捉向均衡价格的有序移动，主要是为了衡量弹性
# 市场效率系数（MEC）用于衡量价格变动的连续性：
# MEC = Var(R_t) / N*Var(r_t)

# Var（Rt）是对数长周期收益的方差，Var（Rt）是对数短周期收益的方差，N是每个长周期内的短周期数。

# 在更具弹性的市场中，这一比率往往接近但略低于1。低市场弹性意味着过度的短期波动，而价格四舍五入、利差和不准确的价格发现等因素将推动MEC远低于1。
# 做市商干预和价格决定不准确后对新信息的部分调整等因素将抑制短期波动，导致MEC升至1以上。

In [ ]:
# 市场调整度量
# 市场调整措施试图区分流动性引起的价格变动和一般市场状况或新信息到达等因素引起的价格变动。当新的信息可用时，即使是小的交易量也可能与大的价格变动相关联。

# 在股票市场上，经常根据资本资产定价模型（CAPM）区分系统性风险和非系统性风险，CAPM也提供了一种提取市场波动的方法。
# 系统性风险不能分散，因为它影响到所有证券。系统风险的程度称为股票的β，它是指股票的日收益率Ri与市场Rm的日收益率Ri的回归系数：
# Ri= α + β*R_m+ui

# 然后，使用回归残差ui将其方差与交易额的每日百分比变化联系起来

# ui^2 = γi + γ2*ΔVi+ ei
# 经市场调整的流动性利用资产回报率与市场回报率之间的回归残差（从而将其从系统风险中清除）来确定资产的内在流动性。
# γ2越小，交易量对资产价格波动性的影响越小，这意味着资产是流动的。系数越低，市场的宽度就越大。

# 没有一个单一的衡量标准明确地衡量紧密性、即时性、深度、广度和弹性。
# 然而，从从业者的角度来看，基于交易难度的证券聚类是可能的，这取决于许多因素，但主要取决于波动性、盘口价差、价格、排队规模、成交量等。



In [3]:
from tools.Cacher import (CACHE_TYPE, save_cache,load_cache_adv,load_cache)

tail = True
if tail:
    file = 'all_tail_qfq'
else:
    file = 'all_train_qfq'
    
df_all = load_cache(file,cache_type=CACHE_TYPE.STOCK).sort_index()
smpl.optimize_data_type(df_all)

data_ = df_all

year1 = '2022'
year2 = '2022'
data_ = df_all.loc[year1+'-01-03':year2+'-12-31']
codes = data_.index.get_level_values(1).unique()
dates = data_.index.get_level_values(0).unique()


# ret = smpl.get_forward_return(data_,'close')
# ret.name = 'ret_forward'
# ret[np.abs(ret)>0.15]=np.nan

In [9]:
from base.JuUnits import excute_for_multidates

x = excute_for_multidates(data_, lambda x:x.shape[0],level='code')


In [15]:
x[x<60].index

Index(['001223', '001256', '001298', '001299', '001300', '001301', '001322',
       '001333', '001338', '601022', '601136', '603052', '603130', '603151',
       '603163', '603280'],
      dtype='object', name='code')

In [18]:
exclude_new = data_.index.get_level_values(1).difference(x[x<60].index)

In [24]:
excluded = data_.loc[(slice(None),exclude_new),]
# data_.loc[(slice(None),'000001'),]

In [25]:
excluded.loc[(slice(None),'000001'),]

open       high        low      close     volume  \
date       code                                                            
2022-01-04 000001  15.799133  15.971697  15.511528  15.971697  1169259.0   
           000002  18.345314  19.437185  18.222948  19.286581  1947202.0   
           000004  19.410000  20.150000  19.410000  19.990000    46185.0   
           000006   4.236619   4.331824   4.198537   4.312783   127024.0   
           000007   6.250000   6.260000   6.100000   6.230000    60056.0   
...                      ...        ...        ...        ...        ...   
2022-12-30 605580   8.150633   8.280879   8.088938   8.274024     7771.0   
           605588  28.522758  28.732264  28.373110  28.732264     2557.0   
           605589  21.483000  21.651300  20.770201  21.176100    77000.0   
           605598  25.299999  25.500000  24.670000  24.900000    22984.0   
           605599   9.579020   9.828823   9.511765   9.780785    65106.0   

                         amount       adj        shares       lshares  \
date       code                                                         
2022-01-04 000001  1.918887e+09  0.958685  1.940592e+06  1.940552e+06   
           000002  3.956476e+09  0.941268  1.161773e+06  9.717553e+05   
           000004  9.200163e+07  1.000000  1.560030e+04  1.163308e+04   
           000006  5.710716e+07  0.952049  1.349995e+05  1.349995e+05   
           000007  3.722346e+07  1.000000  2.309654e+04  2.059654e+04   
...                         ...       ...           ...           ...   
2022-12-30 605580  9.326484e+06  0.685503  2.000000e+04  5.111000e+03   
           605588  7.322457e+06  0.997648  7.309956e+03  2.309956e+03   
           605589  1.638851e+08  0.990000  7.828768e+04  6.152118e+04   
           605598  5.754628e+07  1.000000  1.727435e+04  5.264347e+03   
           605599  6.582705e+07  0.960784  7.777778e+04  5.866778e+04   

                   market_value  liquidity_market_value industry  
date       code                                                   
2022-01-04 000001  3.099454e+11            3.099391e+11    股份制银行  
           000002  2.240663e+11            1.874184e+11     住宅开发  
           000004  3.118501e+09            2.325452e+09     基础软件  
           000006  5.822235e+09            5.822235e+09     住宅开发  
           000007  1.438914e+09            1.283164e+09   商业物业经营  
...                         ...                     ...      ...  
2022-12-30 605580  1.654805e+09            4.228854e+08     热力服务  
           605588  2.100316e+09            6.637027e+08     光学元件  
           605589  1.657828e+10            1.302779e+10     合成树脂  
           605598  4.301313e+09            1.310822e+09   其他专业工程  
           605599  7.607277e+09            5.738169e+09     钟表珠宝  

[735639 rows x 12 columns]